# 🔄 Swaps: Interest Rate and Currency Swaps

## Introduction
This notebook provides an in-depth exploration of **swaps**, as presented in Chapter 7 of John Hull's *Options, Futures, and Other Derivatives*. We will cover the key concepts, pricing mechanisms, and applications of **interest rate swaps** and **currency swaps**.

## 🏦 Interest Rate Swaps
An **interest rate swap** involves exchanging cash flows between two parties, typically swapping fixed-rate payments for floating-rate payments or vice versa. The most common type is the **plain vanilla interest rate swap**, where one party pays a fixed rate while the other pays a floating rate based on a reference rate like LIBOR.

The general cash flow structure of an interest rate swap is:

- **Fixed payment leg**: The party making fixed payments will pay a fixed interest rate, calculated on a notional principal.
  
- **Floating payment leg**: The party making floating payments will pay interest based on a reference rate (e.g., LIBOR) adjusted periodically.

### 📊 Fixed and Floating Cash Flows
- **Fixed Cash Flow**:
  $$ 
  \text{Fixed Cash Flow} = \text{Notional Principal} \times \frac{\text{Fixed Rate}}{\text{Payment Frequency}} 
  $$

- **Floating Cash Flow**:
  $$ 
  \text{Floating Cash Flow} = \text{Notional Principal} \times \frac{\text{Floating Rate}}{\text{Payment Frequency}} 
  $$

## 💱 Currency Swaps
In a **currency swap**, two parties exchange principal and interest payments in different currencies. The typical structure involves one party paying interest in one currency, while the other party pays interest in a different currency. At the swap's initiation, the principal amounts are exchanged, and at maturity, they are swapped back.

### 🧮 Valuing a Currency Swap
The value of a currency swap is determined by the present value of future cash flows in each currency. The steps include:
1. **Discounting the foreign currency cash flows** at the foreign interest rate.
2. **Converting** the foreign currency present value into the domestic currency using the current exchange rate.
3. **Discounting the domestic cash flows** at the domestic interest rate.

The net value of the swap is the difference between the present values of the cash flows.

## 🛠️ Applications of Swaps
Swaps are widely used for hedging, arbitrage, and speculation in financial markets. Some common applications include:
- **Managing interest rate exposure**: Companies use interest rate swaps to convert floating-rate debt to fixed-rate debt (or vice versa) to manage their interest rate risk.
- **Foreign exchange management**: Multinational corporations utilize currency swaps to hedge foreign currency cash flows and minimize foreign exchange risk.


In [1]:
import pandas as pd 
import numpy as np

In [2]:
"""
Example : Swap with fixed rate and floating rate (APPLE receives fixed rate and pays floating rate to Citigroup).
"""
# ======= I. Set the contract details =======
principal = 1e9
fixed_rate = 0.03 / 4 # 3% annual rate, quarterly payments
SOFR = [0.022 / 4, 0.026 / 4] # 2.2% and 2.6% annual rate, quarterly payments 

# ======= II. Calculate the payments =======
apple_payment = principal * fixed_rate
citigroup_payment = principal * SOFR[0]

difference = apple_payment - citigroup_payment
print(f"APPLE pays {apple_payment}$ to Citigroup and Citigroup pays {citigroup_payment} to Apple for the first quarter.")
print(f"This results in a net flow of {difference}$ from APPLE to Citigroup.")

# ======= II.bis Simplifying the calculation =======
flow_1 = principal * (SOFR[0] - fixed_rate)
flow_2 = principal * (SOFR[1] - fixed_rate)

print(f"\nAAPLE receives at first quarter: {flow_1}$ and at second quarter: {flow_2}$ from Citigroup.")

APPLE pays 7500000.0$ to Citigroup and Citigroup pays 5500000.0 to Apple for the first quarter.
This results in a net flow of 2000000.0$ from APPLE to Citigroup.

AAPLE receives at first quarter: -2000000.0$ and at second quarter: -1000000.0$ from Citigroup.


---
## ***Problems & Exercises***

In [3]:
"""
7.11
Interest rate swap between two companies.
A wants to receive a floating rate and B wants to receive a fixed rate.
"""
# ======= I. Set the details =======
sofr = np.random.uniform(0.02, 0.05)

principal = 20e6
offered_rates = pd.DataFrame({
    "Fixed Rate": [0.05, 0.064],
    "Floating Rate": [sofr + 0.001, sofr + 0.006]
})

# ======= II. Evaluate if the situation can provide a advantage =======
fixed_difference = abs(offered_rates["Fixed Rate"][0] - offered_rates["Fixed Rate"][1])
floating_difference = abs((offered_rates["Floating Rate"][0] - sofr) - (offered_rates["Floating Rate"][1] - sofr))

opportunity = fixed_difference - floating_difference
if opportunity > 0:
    print(f"The situation presents an opportunity for a swap of {opportunity * 100:.2f}%.")

# As fixed rate is higher than floating rate, A will pay less with the fixed rate and B will pay less with the floating rate.
# Therefore, A will receive the fixed rate and B will receive the floating rate.

The situation presents an opportunity for a swap of 0.90%.


In [4]:
"""
7.12
Pricing a swap between fixed_rate and LIBOR 6 months.
"""
# ======= I. Set the details =======
principal = 100e6
fixed_rate = 0.04 # 4% annual rate, semestrial payments
forward_LIBOR = 0.03 # 3% annual rate
six_month_LIBOR = 0.024 # 2.4% annual rate, listed 2 months ago
ois_rate = 0.027 # 2.7% annual rate, continuous payments

# ======= II. Calculate the payments =======
"""
There will be two payments for this swap (semestrial payments), the first take place in 4 months and the second in 10 months.
"""
# First payment
fixed_payment_1 = principal * (fixed_rate/2) 
floating_payment_1 = principal * (six_month_LIBOR/2) # We use the 6 months LIBOR rate listed 2 months ago
present_value_exchange_1 = (fixed_payment_1 - floating_payment_1) * np.exp(-ois_rate * 4/12)

# Second payment
fixed_payment_2 = principal * (fixed_rate/2)
floating_payment_2 = principal * (forward_LIBOR/2) # The 6 months LIBOR rate is not known yet for the second payment so we use the forward rate
present_value_exchange_2 = (fixed_payment_2 - floating_payment_2) * np.exp(-ois_rate * 10/12)

# ======= III. Calculate the present value of the swap =======
present_value_swap = present_value_exchange_1 + present_value_exchange_2
print(f"The present value of the swap for the party that pays the floating rate is {present_value_swap:.2f}$.")
print(f"The present value of the swap for the party that pays the fixed rate is {-present_value_swap:.2f}$.")

The present value of the swap for the party that pays the floating rate is 1281707.92$.
The present value of the swap for the party that pays the fixed rate is -1281707.92$.


In [5]:
"""
7.13
Creating a currency swap for a two companies.
"""
# ======= I. Set the details =======
principal = 100
data = pd.DataFrame({
    "Yen": [0.05, 0.065],
    "USD": [0.096, 0.10],
}, index=["Company X", "Company Y"])

# ======= II. Evaluate if the situation can provide a advantage =======
yen_difference = abs(data["Yen"][0] - data["Yen"][1])
usd_difference = abs(data["USD"][0] - data["USD"][1])

opportunity = yen_difference - usd_difference # If positive, it means that there is an opportunity for a swap
if opportunity > 0:
    print(f"The situation presents an opportunity for a swap of {opportunity * 100:.2f}%.")
    print("Company X will receive Yen and Company Y will receive USD.")

The situation presents an opportunity for a swap of 1.10%.
Company X will receive Yen and Company Y will receive USD.


In [6]:
"""
7.14
Pricing a swap on currency exchange.
"""
# ======= I. Set the details of the swap =======
maturity = 15/12 # 15 months

principal_GBP = 20e6 
principal_USD = 30e6

swap_rate_GBP = 0.1 # rates 
swap_rate_USD = 0.06

# ======= II. Today's parameters =======
rate_GBP = 0.07
rate_USD = 0.04
usd_gbp = 1.55

# ======= III. Calculate the payments =======
pv_payment1_GBP = principal_GBP * swap_rate_GBP * np.exp(-rate_GBP * 3/12)
pv_payment2_GBP = (principal_GBP * swap_rate_GBP + principal_GBP) * np.exp(-rate_GBP * maturity)
pv_payment_GBP = pv_payment1_GBP + pv_payment2_GBP
print(f"The present value of the payments in GBP is {pv_payment_GBP:.2f}£.")

pv_payment1_USD = principal_USD * swap_rate_USD * np.exp(-rate_USD * 3/12)
pv_payment2_USD = (principal_USD * swap_rate_USD + principal_USD) * np.exp(-rate_USD * maturity)
pv_payment_USD = pv_payment1_USD + pv_payment2_USD
print(f"The present value of the payments in USD is {pv_payment_USD:.2f}$.")

swap_price = pv_payment_USD - pv_payment_GBP * usd_gbp
print(f"The price of the swap for the party who pays the GBP is {swap_price:.2f}$.")

The present value of the payments in GBP is 22122119.65£.
The present value of the payments in USD is 32031185.40$.
The price of the swap for the party who pays the GBP is -2258100.05$.


In [9]:
"""
7.19
Supposing that one party does default, how to calculate the value of the swap.
"""
principal = 10e6
maturity = 5 # with semestrial payments
fixed_rate = 0.04
libor_6m = 0.03 # At the middle of the 3rd year
ois_rate = 0.018
forward_rate = 0.02 # At the end of the 3rd year

# ======= I. Calculate the payments =======
# At the end of year 3 
fixed_payment = principal * (fixed_rate/2)
floating_payment = principal * (libor_6m/2)
present_value_exchange = (fixed_payment - floating_payment) # Evaluated at the day of default

# Other payments
fixed_payments = [principal * (fixed_rate/2) for _ in range((5-3)*2)]
floating_payments = [principal * (forward_rate/2) for _ in range((5-3)*2)]
present_value_exchange_other = sum([(fixed_payments[i] - floating_payments[i]) * np.exp(-ois_rate * (i+1)/2) for i in range((5-3)*2)])

# ======= II. Calculate the cost of default =======
default_cost = present_value_exchange + present_value_exchange_other
print(f"The cost of default is {default_cost:.2f}$.")

The cost of default is 441120.29$.
